# Introduction
- Recurrent neural networks (RNN) are a class of neural networks that is powerful for modeling sequence data such as time series or natural language.
- Schematically, a RNN layer uses a `for` loop to iterate over the timesteps of a sequence, while maintaining an internal state that encodes information about the timesteps it has been so far.
- The Keras RNN API is designed with a focus on:
    - Ease of use
        - The built-in `keras.layers.RNN`, `keras.layers.LSTM`, and `keras.layers.GRU` layers enable you to quickly build recurrent models without having to make difficult configuration choices.
    - Ease of customization
        - You can also define your own RNN cell layer (the inner part of the `for` loop) with custom behavior, and use it with the generic `keras.layers.RNN` layer (the `for` loop itself).
        - This allows you to quickly prototype different research ideas in a flexible way with minimal code.

# Setup

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# Built-in RNN layers: a simple example
- Here is a simple example of a `Sequential` model that processes sequences of integers, emdes each integer into a 64-dimensional vector, and then proceses the sequence of vectors using a `LSTM` layer.

In [2]:
model = keras.Sequential()

# Add an Embedding layer expecting vocab of size 1000
# Output embedding dimension of size 64
model.add(layers.Embedding(input_dim=1000, output_dim=64))

# Add a LSTM layer with 128 internal units
model.add(layers.LSTM(128))

# Add a Dense layer with 10 units
model.add(layers.Dense(10))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 64)          64000     
_________________________________________________________________
lstm (LSTM)                  (None, 128)               98816     
_________________________________________________________________
dense (Dense)                (None, 10)                1290      
Total params: 164,106
Trainable params: 164,106
Non-trainable params: 0
_________________________________________________________________


# Outputs and states
- By default, the output of a RNN layer contain a single vector per sample. 
    - This vector is the RNN cell output corresponding to the last timestep, containing information about the entire input sequence. 
        - The shape of this output is `(batch_size, units)` where units corresponds to the units argument passed to the layer's constructor.
    - A RNN layer can also return the entire sequence of outputs for each sample (one vector per timestep per sample), if you set `return_sequences=True`. 
        - The shape of this output is `(batch_size, timesteps, units)`.

In [3]:
model = keras.Sequential()
model.add(layers.Embedding(input_dim=1000, output_dim=64))

# The output of the following GRU layer will be a 3D tensor of shape (batch_size, timesteps, 256)
model.add(layers.GRU(256, return_sequences=True))

# The output of the following SimpleRNN layer will be a 2D tensor of shape (batch_size, 128)
model.add(layers.SimpleRNN(128))

model.add(layers.Dense(10))

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, None, 64)          64000     
_________________________________________________________________
gru (GRU)                    (None, None, 256)         247296    
_________________________________________________________________
simple_rnn (SimpleRNN)       (None, 128)               49280     
_________________________________________________________________
dense_1 (Dense)              (None, 10)                1290      
Total params: 361,866
Trainable params: 361,866
Non-trainable params: 0
_________________________________________________________________


- In addition, a RNN layer can return its final internal state(s). 
    - The returned states can be used to resume the RNN execution later, or to initialize another RNN. 
    - This setting is commonly used in the encoder-decoder sequence-to-sequence model, where the encoder final state is used as the initial state of the decoder.
- To configure a RNN layer to return its internal state, set the `return_state=True` when creating the layer. 
    - Note that LSTM has 2 state tensors, but GRU only has one.
- To configure the initial state of the layer, just call the layer with additional keyword argument `initial_state`. 
    - Note that the shape of the state needs to match the unit size of the layer, like in the example below.

In [4]:
encoder_vocab = 1000
decoder_vocab = 2000

encoder_input = layers.Input(shape=(None,))
encoder_embedded = layers.Embedding(input_dim=encoder_vocab, output_dim=64)(encoder_input)

# Return states in addition to output
output, state_h, state_c = layers.LSTM(64, return_state=True, name='encoder')(encoder_embedded)

encoder_state = [state_h, state_c]

In [5]:
decoder_input = layers.Input(shape=(None,))
decoder_embedded = layers.Embedding(input_dim=decoder_vocab, output_dim=64)(decoder_input)

# Pass the 2 states to a new LSTM layer as the initial state
decoder_output = layers.LSTM(64, name='decoder')(decoder_embedded, initial_state=encoder_state)

output = layers.Dense(10)(decoder_output)

In [6]:
model = keras.Model([encoder_input, decoder_input], output)
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, None, 64)     64000       input_1[0][0]                    
__________________________________________________________________________________________________
embedding_3 (Embedding)         (None, None, 64)     128000      input_2[0][0]                    
______________________________________________________________________________________________

# RNN layers and RNN cells
- In addition to the built-in RNN layers, the RNN API also provides cell-level APIs.
    - Unlike RNN layers, which processes whole batches of input sequences, the RNN cell only processes a single timestep.
- The cell is the inside of the `for` loop of a RNN layer.
    - Wrapping a cell inside a `keras.layers.RNN` layer gives you a layer capable of processing batches of sequencs.
    - E.g. `RNN(LSTMCell(10)`
- There are three built-in RNN cells, each corresponding to the matching RNN layer.
    - `keras.layers.SimpleRNNCell` corresponds to the `SimpleRNN` layer.
    - `keras.layers.GRUCell` corresponds to the `GRU` layer.
    - `keras.layers.LSTMCell` corresponds to the `LSTM` layer.

# Cross-batch statefulness
- When processing very long sequences (possibly infinite), you may want to use the pattern of **cross-batch statefulness**.
- Normally, the internal state of a RNN layer is reset every time it sees a new batch (i.e. every sample seen by the layer is assume to be independent from the past). 
    - The layer will only maintain a state while processing a given sample.
- If you have very long sequences though, it is useful to **break them into shorter sequences, and to feed these shorter sequences sequentially into a RNN layer without resetting the layer's state**. 
    - That way, the layer can retain information about the entirety of the sequence, even though it's only seeing one sub-sequence at a time.
- You can do this by setting `stateful=True` in the constructor.
- When you want to clear the state, you can use `layer.reset_states()`.

In [7]:
paragraph1 = np.random.random((20, 10, 50)).astype(np.float32)
paragraph2 = np.random.random((20, 10, 50)).astype(np.float32)
paragraph3 = np.random.random((20, 10, 50)).astype(np.float32)

In [8]:
lstm_layer = layers.LSTM(64, stateful=True)
output = lstm_layer(paragraph1)
output = lstm_layer(paragraph2)
output = lstm_layer(paragraph3)

In [9]:
# reset_states() will reset the cached state to the original initial_state.
# If no initial_state was provided, zero-states will be used by default.
lstm_layer.reset_states()

# RNN State Reuse
- The recorded states of the RNN layer are not included in the `layer.weights()`. 
- If you would like to reuse the state from a RNN layer, you can retrieve the states value by **`layer.states`** and use it as the initial state for a new layer via the Keras functional API like `new_layer(inputs, initial_state=layer.states)`, or model subclassing.
- Please also note that sequential model might not be used in this case since it only supports layers with single input and output, the extra input of initial state makes it impossible to use here.

In [10]:
paragraph1 = np.random.random((20, 10, 50)).astype(np.float32)
paragraph2 = np.random.random((20, 10, 50)).astype(np.float32)
paragraph3 = np.random.random((20, 10, 50)).astype(np.float32)

lstm_layer = layers.LSTM(64, stateful=True)
output = lstm_layer(paragraph1)
output = lstm_layer(paragraph2)

In [11]:
existing_state = lstm_layer.states

In [12]:
new_lstm_layer = layers.LSTM(64)
new_output = new_lstm_layer(paragraph3, initial_state=existing_state)

# Bidirectional RNNs
- For sequences other than time series (e.g. text), it is often the case that a RNN model can perform better if it not only processes sequence from start to end, but also backwards. 
- For example, to predict the next word in a sentence, it is often useful to have the context around the word, not only just the words that come before it.
- Keras provides an easy API for you to build such bidirectional RNNs: the `keras.layers.Bidirectional` wrapper.
    - Under the hood, `Bidirectional` will copy the RNN layer passed in, and flip the `go_backwards` field of the newly copied layer, so that it will process the inputs in reverse order.
    - The output of the `Bidirectional` RNN will be, by default, the sum of the forward and backward layer output.
        - If you need a different merging behavior, e.g. concatenation, change the `merge_mode` parameter in the `Bidirectional` wrapper constructor.

In [13]:
model = keras.Sequential()

model.add(
    layers.Bidirectional(layers.LSTM(64, return_sequences=True), input_shape=(5, 10))
)

model.add(layers.Bidirectional(layers.LSTM(32)))
model.add(layers.Dense(10))

model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional (Bidirectional (None, 5, 128)            38400     
_________________________________________________________________
bidirectional_1 (Bidirection (None, 64)                41216     
_________________________________________________________________
dense_3 (Dense)              (None, 10)                650       
Total params: 80,266
Trainable params: 80,266
Non-trainable params: 0
_________________________________________________________________


# Performance optimization and CuDNN kernels
- In TensorFlow 2.0, the **built-in LSTM and GRU layers have been updated to leverage CuDNN kernels by default** when a GPU is available. 
    - With this change, the prior `keras.layers.CuDNNLSTM/CuDNNGRU` layers have been deprecated, and you can build your model without worrying about the hardware it will run on.
- However, a layer will not be able to use the CuDNN kernel if you change the defualts of the built-in LSTM or GRU layers, such as the following.
    - Changing the activation function from `tanh` to something else
    - Changing the `recurrent_activation` function from `sigmoid` to something else
    - Using `recurrent_dropout` > 0
    - Setting `unroll` to `True`, which forces LSTM/GRU to decompose the inner `tf.while_loop` into an unrolled for loop
    - Setting `use_bias` to `False`
    - Using masking when the input data is not strictly right padded (if the mask corresponds to strictly right padded data, CuDNN can still be used. This is the most common case).

## Using CuDNN kernels when available
- Let's build a simple LSTM model to demonstrate the performance difference.
- We'll use sequences of sequence of rows of MNIST digits (treating each row of pixels as a timestep) as inputs, and we'll predict the digits' labels.

In [14]:
batch_size = 64

# Each MNIST image batch is a tensor of shape (batchbatch_size, 28, 28).
# Each input sequence will be of size (28, 28).
input_dim = 28

units = 64
output_size = 10 # labels are from 0 to 9

# Build the RNN model
def build_model(allow_cudnn_kernel=True):
    # CuDNN is only available at the layer level, and not at the cell level.
    # This means `LSTM(units)` will use the CuDNN kernel, while RNN(LSTMCell(units)) will run on non-CuDNN kernel.
    if allow_cudnn_kernel:
        # The LSTM layer with default options uses CuDNN
        lstm_layer = layers.LSTM(units, input_shape=(None, input_dim))
    else:
        # Wrapping a LSTMCell in a RNN layer will not leverage CuDNN
        lstm_layer = layers.RNN(
            layers.LSTMCell(units), 
            input_shape=(None, input_dim)
        )
        
    model = keras.models.Sequential([
        lstm_layer,
        layers.BatchNormalization(),
        layers.Dense(output_size)
    ])
    
    return model

- Let's load the MNIST dataset.

In [15]:
mnist = keras.datasets.mnist

(X_train, y_train), (X_test, y_test) = mnist.load_data()
X_train, X_test = X_train / 255.0, X_test / 255.0
sample, sample_label = X_train[0], y_train[0]

- Let's create a model instance and train it.
- We choose `SparseCategoricalCrossentropy` as the loss function.

In [16]:
model = build_model(allow_cudnn_kernel=True)

model.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer='sgd',
    metrics=['accuracy']
)

In [17]:
model.fit(X_train, y_train, validation_data=(X_test, y_test),
         batch_size=batch_size, epochs=1)

Train on 60000 samples, validate on 10000 samples
60000/60000 [==============================] - 68s 1ms/sample - loss: 0.9511 - accuracy: 0.7014 - val_loss: 0.5794 - val_accuracy: 0.8083


- Now, let's compare to a model that does not use the CuDNN kernel.

In [18]:
noncudnn_model = build_model(allow_cudnn_kernel=False)
noncudnn_model.set_weights(model.get_weights())
noncudnn_model.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer="sgd",
    metrics=["accuracy"],
)
noncudnn_model.fit(X_train, y_train, validation_data=(X_test, y_test),
         batch_size=batch_size, epochs=1)

Train on 60000 samples, validate on 10000 samples
60000/60000 [==============================] - 70s 1ms/sample - loss: 0.3739 - accuracy: 0.8880 - val_loss: 0.5675 - val_accuracy: 0.8139


- When running on a machine with a NVIDIA GPU and CuDNN installed, the model built with CuDNN is much faster to train compared to the model that use the regular TensorFlow kernel.
- The same CuDNN-enabled model can also be use to run inference in a CPU-only environment. 
    - The model will run on CPU by default if no GPU is available.

# RNNs with list/dict inputs or nested inputs
- Nested structures allow implementers to include more information within a single timestep. 
    - For example, a video frame could have audio and video input at the same time. 
        - The data shape in this case could be:`[batch, timestep, {"video": [height, width, channel], "audio": [frequency]}]`.
    - In another example, handwriting data could have both coordinates x and y for the current position of the pen, as well as pressure information. 
        - So the data representation could be:`[batch, timestep, {"location": [x, y], "pressure": [force]}]`.
- The following code provides an example of how to build a custom RNN cell that accepts such structured inputs.

In [19]:
class NestedCell(keras.layers.Layer):
    def __init__(self, unit_1, unit_2, unit_3, **kwargs):
        self.unit_1 = unit_1
        self.unit_2 = unit_2
        self.unit_3 = unit_3
        self.state_size = [tf.TensorShape([unit_1]), tf.TensorShape([unit_2, unit_3])]
        self.output_size = [tf.TensorShape([unit_1]), tf.TensorShape([unit_2, unit_3])]
        super(NestedCell, self).__init__(**kwargs)

    def build(self, input_shapes):
        # expect input_shape to contain 2 items, [(batch, i1), (batch, i2, i3)]
        i1 = input_shapes[0][1]
        i2 = input_shapes[1][1]
        i3 = input_shapes[1][2]

        self.kernel_1 = self.add_weight(
            shape=(i1, self.unit_1), initializer="uniform", name="kernel_1"
        )
        self.kernel_2_3 = self.add_weight(
            shape=(i2, i3, self.unit_2, self.unit_3),
            initializer="uniform",
            name="kernel_2_3",
        )

    def call(self, inputs, states):
        # inputs should be in [(batch, input_1), (batch, input_2, input_3)]
        # state should be in shape [(batch, unit_1), (batch, unit_2, unit_3)]
        input_1, input_2 = tf.nest.flatten(inputs)
        s1, s2 = states

        output_1 = tf.matmul(input_1, self.kernel_1)
        output_2_3 = tf.einsum("bij,ijkl->bkl", input_2, self.kernel_2_3)
        state_1 = s1 + output_1
        state_2_3 = s2 + output_2_3

        output = (output_1, output_2_3)
        new_states = (state_1, state_2_3)

        return output, new_states

    def get_config(self):
        return {"unit_1": self.unit_1, "unit_2": unit_2, "unit_3": self.unit_3}

- Let's build a Keras model that uses a `keras.layers.RNN` layer and the custom cell defined above.

In [20]:
unit_1 = 10
unit_2 = 20
unit_3 = 30

i1 = 32
i2 = 64
i3 = 32
batch_size = 64
num_batches = 10
timestep = 50

cell = NestedCell(unit_1, unit_2, unit_3)
rnn = keras.layers.RNN(cell)

input_1 = keras.Input((None, i1))
input_2 = keras.Input((None, i2, i3))

outputs = rnn((input_1, input_2))

model = keras.models.Model([input_1, input_2], outputs)

model.compile(optimizer="adam", loss="mse", metrics=["accuracy"])

- Let's train the model with randomly generated data.

In [21]:
input_1_data = np.random.random((batch_size * num_batches, timestep, i1))
input_2_data = np.random.random((batch_size * num_batches, timestep, i2, i3))
target_1_data = np.random.random((batch_size * num_batches, unit_1))
target_2_data = np.random.random((batch_size * num_batches, unit_2, unit_3))
input_data = [input_1_data, input_2_data]
target_data = [target_1_data, target_2_data]

model.fit(input_data, target_data, batch_size=batch_size)

Train on 640 samples
640/640 [==============================] - 7s 11ms/sample - loss: 0.7538 - rnn_1_loss: 0.2654 - rnn_1_1_loss: 0.4884 - rnn_1_accuracy: 0.1266 - rnn_1_1_accuracy: 0.0318
